In [20]:
# example of calculating the frechet inception distance in Keras
import os
import numpy
import cv2
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.datasets.mnist import load_data
from skimage.transform import resize

In [25]:
def get_images(imageroot):
    images = []
    for filename in os.listdir(imageroot):
        img = cv2.imread(os.path.join(imageroot, filename))
        if img is not None:
            images.append(img)
    ret = np.asarray(images)
    print('Data loaded, shape:', ret.shape)
    return ret

def get_images_gan(imageroot):
    images = []
    for filename in os.listdir(imageroot):
        if filename.endswith('fake_B.png'):
            img = cv2.imread(os.path.join(imageroot, filename))
            if img is not None:
                images.append(img)
    ret = np.asarray(images)
    print('Data loaded, shape:', ret.shape)
    return ret

# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [26]:
CARTOON_DIR = 'face_cartoon'
LS_DIR = 'cart_cyclegan_ls/test_latest/images'
VAN_DIR = 'cart_cyclegan_vanilla/test_latest/images'
W_DIR = 'cart_cyclegan_wgan/test_latest/images'

In [27]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
# define two fake collections of images
images1 = (get_images(CARTOON_DIR))[:1000]
images2 = get_images_gan(LS_DIR)
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

Data loaded, shape: (5000, 256, 256, 3)
Data loaded, shape: (1894, 256, 256, 3)
Prepared (1000, 256, 256, 3) (1894, 256, 256, 3)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Scaled (1000, 299, 299, 3) (1894, 299, 299, 3)
FID (same): -0.000
FID (different): 58.113


In [31]:
# define two fake collections of images
images2 = get_images_gan(VAN_DIR)
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images2 = images2.astype('float32')
# resize images
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images2 = preprocess_input(images2)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

Data loaded, shape: (1894, 256, 256, 3)
Prepared (1000, 299, 299, 3) (1894, 256, 256, 3)
Scaled (1000, 299, 299, 3) (1894, 299, 299, 3)
FID (different): 58.803


In [32]:
# define two fake collections of images
images2 = get_images_gan(W_DIR)
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images2 = images2.astype('float32')
# resize images
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images2 = preprocess_input(images2)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

Data loaded, shape: (1894, 256, 256, 3)
Prepared (1000, 299, 299, 3) (1894, 256, 256, 3)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Scaled (1000, 299, 299, 3) (1894, 299, 299, 3)
FID (different): 138.299
